In [1]:
import pandas as pd
import os

# pd.options.display.max_rows = 200
# pd.options.display.max_columns = 10000

In [2]:
# change campus names
def change_school_name(df):
    
    # school name dictionary
    school_dict = {
        'Bilingual': '',
        'Non-LEP DL': '',
        'NON-LEP DL': '',
        'LEP DL BIL': '',
        'ESL': '',
        'ESOL': '',
        'Regular': '',
        'Self Contained': '',
        
        'Mt.': 'Mountain',
        'Tr.': 'Trail',
        
        'MIS': '',
        'ES': '',
        'MS': '',
        'HS': '',
        'High School': '',
        'Elementary School': '',
        'Middle School': '',
        'Elem School': '',
        'Elementary': '',
        'Elem: ': '',
        'EC': 'Early College',
        'Wm S Lott Juvenile Ctr': 'LOTT',
        'Wm. Lott': 'LOTT',
        'Lott': 'LOTT',
        'RROC': 'Round Rock Opportunity Center',
        'Daep': 'DAEP',
        'Deepwood': 'Deep Wood',
        'Liveoak': 'Live Oak',
        'Joe Lee': '',
        'Xenia': '',
        'Patsy': '',
        'Neysa': '',
        'Noel': '',
        'C.D.': 'CD',
        'C. D.': 'CD',
        'Claude': '',
        'Elsa': '',
        'James': '',
        'Kathy': '',
        'Linda': '',
        'Patsy': '',
        'RRISD': '',
        
    }
    
    # change keys(k) to values(v)
    for k, v in school_dict.items():
        df['School'] = df['School'].map(lambda dis_cell: dis_cell.replace(k, v).strip())
    
    return None

In [3]:
school_type = {
    # list of middle schools
    'CD Fulkes': 'MS',
    'Canyon Vista': 'MS',
    'Cedar Valley': 'MS',
    'Chisholm Trail': 'MS',
    'Deerpark': 'MS',
    'Grisham': 'MS',
    'Hernandez': 'MS',
    'Hopewell': 'MS',
    'Pearson Ranch': 'MS',
    'Ridgeview': 'MS',
    'Walsh': 'MS',
    
    # list of high schools
    'Cedar Ridge': 'HS',
    'McNeil': 'HS',
    'Round Rock': 'HS',
    'Stony Point': 'HS',
    'Westwood': 'HS',
    'Success': 'HS',
    'Early College': 'HS',
    
    # other schools
    'Round Rock Opportunity Center': 'Other',
    'DAEP': 'Other',
    'JJAEP': 'Other',
    'LOTT': 'Other',
}

In [5]:
# file directories
# excel files from 2016-08-23 to 2018-05-29
path_one = os.getcwd() + '/data/excel/one/'

# list of dataframes
# each dataframe is one excel file
df_excel = []

grades = ['ECE', 'PK', 'K', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

# file names
filenames_one = os.listdir(path_one)
filenames_one.sort()

for filename in filenames_one:
    df_one = pd.DataFrame()

    # xlsx and xls file only
    if filename.endswith('xlsx' or 'xls'):
        print(filename)
        
        # get the first sheet in an excel file
        temp = pd.read_excel(path_one + filename, header=2)
        temp.fillna(0, inplace=True)
    
        # change column names to strings since grades from 1-12 are int
        temp.columns = [str(x) for x in temp.columns]        

        # school name column 
        df_one['School'] = temp['CAMPUS'].str.strip()
        
        # date column
        # change a filename with '.' instead of '-'
        df_one['Date'] = filename[:10].replace('.', '-')
        
        # change Date column into datetime object
        df_one['Date'] = pd.to_datetime(df_one['Date'])
        
        # create 14 more rows for each school (so we can have 15 grades)
        df_one = df_one.append([df_one]*14, ignore_index=True)

        # sort by school name and reset index
        df_one = df_one.sort_values(by=['School'])
        df_one.reset_index(inplace=True, drop=True)

        for i in range(len(df_one)):
            
            # grades
            idx = i % 15
            df_one.loc[i, 'Grade'] = grades[idx]
        
        df_one['Enrolled'] = 0
        for i in range(len(df_one)):

            # enrollment
            school = df_one['School'][i]
            grade = str(df_one['Grade'][i])
            if not temp[temp['CAMPUS'].str.strip() == school][grade].values.size == 0:
                df_one.loc[i, 'Enrolled'] = temp[temp['CAMPUS'].str.strip() == school][grade].values[0]


        # set all schools to 'Regular' (default)
        # then change types accordingly
        df_one['Group'] = 'Regular'
        
        for i in range(len(df_one)):

            # group
            school = str(df_one.loc[i, 'School']).lower()

            if 'bilingual' in school:
                df_one.loc[i, 'Group'] = 'Bilingual'
            elif 'non-lep dl' in school:
                df_one.loc[i, 'Group'] = 'Non-LEP DL'
            elif 'lep dl bil' in school:
                df_one.loc[i, 'Group'] = 'LEP DL BIL'
            elif 'esol' in school:
                df_one.loc[i, 'Group'] = 'ESL'
            elif 'esl' in school:
                df_one.loc[i, 'Group'] = 'ESL'
            elif 'regular' in school:
                df_one.loc[i, 'Group'] = 'Regular'
            elif 'self contained' in school:
                df_one.loc[i, 'Group'] = 'Self Contained'
        
        df_one.dropna(inplace=True)
        
        # remove rows that aren't school (Total, )
        df_one = df_one[df_one['School'].str.contains('tota|campus', case=False, regex=True) != True]

        # change school names
        df_one['School'] = df_one['School'].str.strip().str.replace('/', ' ')
        change_school_name(df_one)
        
        # reset index for future use
        df_one.reset_index(inplace=True, drop=True)

        # set all schools to 'ES' (default)
        # then change types accordingly
        df_one['Type'] = 'ES'
        for i in range(len(df_one)):
            if df_one.loc[i, 'School'] in school_type.keys():
                df_one.loc[i, 'Type'] = school_type[df_one.loc[i, 'School']]
        
        #append each temp dataframes(sheets) to a list
        df_excel.append(df_one)

df_one_complete = pd.DataFrame(columns=['Date', 'School', 'Grade', 'Enrolled', 'Group', 'Type'])

for i in range(len(df_excel)):
    df_one_complete = df_one_complete.append(df_excel[i], sort=False, ignore_index=True)

2016-08-23 enrollment.xls.xlsx
2016-08.30 enrollment.xls.xlsx
2016-09-06 enrollment.xls.xlsx
2016-09-13 enrollment.xls.xlsx
2016-09-20 enrollment.xls.xlsx
2016-09-27 enrollment.xls.xlsx
2016-10-04 enrollment.xls.xlsx
2016-10-11 enrollment.xls.xlsx
2016-10-18 enrollment.xls.xlsx
2016-10-25 enrollment.xls.xlsx
2016-11-01 enrollment.xls.xlsx
2016-11-08 enrollment.xls.xlsx
2016-11-15 enrollment.xls.xlsx
2016-11-29 enrollment.xls.xlsx
2016-12-06 enrollment.xls.xlsx
2016-12-13 enrollment.xls.xlsx
2017-01-10 enrollment.xls.xlsx
2017-01-17 enrollment.xls.xlsx
2017-01-24 enrollment.xls.xlsx
2017-01-31 enrollment.xls.xlsx
2017-02-07 enrollment.xls.xlsx
2017-02-14 enrollment.xls.xlsx
2017-02-21 enrollment.xls.xlsx
2017-02-28 enrollment.xls.xlsx
2017-03-07 enrollment.xls.xlsx
2017-03-21 enrollment.xls.xlsx
2017-03-28 enrollment.xls.xlsx
2017-04-04 enrollment.xls.xlsx
2017-04-11 enrollment.xls.xlsx
2017-04-18 enrollment.xls.xlsx
2017-04-25 enrollment.xls.xlsx
2017-05-02 enrollment.xls.xlsx
2017-05-

In [6]:
group_dict = {
    'Bil': 'Bilingual',
    'DL': 'Non-LEP DL',
    'Reg': 'Regular'
}

In [7]:
grades_dict = {
    'EE': 'ECE', 
    'KG': 'K',
    '01': '1',
    '02': '2', 
    '03': '3', 
    '04': '4', 
    '05': '5', 
    '06': '6', 
    '07': '7', 
    '08': '8', 
    '09': '9', 

}

In [8]:
# file directories
# excel files after 2018-05-29 ~ 11/16
path_two = os.getcwd() + '/data/excel/two/'

# list of dataframes
# each dataframe is one excel file
df_excel = []

grades = ['EE', 'PK', 'KG', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# file names
filenames_two = os.listdir(path_two)
filenames_two.sort()

for filename in filenames_two:
    df_two = pd.DataFrame()

    # xlsx and xls file only
    if filename.endswith('xlsx' or 'xls'):
        print(filename)
        
        # get the first sheet in an excel file
        temp = pd.read_excel(path_two + filename, header=2)
        temp.iloc[:, 0].fillna(method='ffill', inplace=True)
        temp.iloc[:, 1:16].fillna(0, inplace=True)

        df_two = pd.DataFrame()

        # school name column 
        df_two['School'] = temp.iloc[:, 0].str.strip()

        # date column
        df_two['Date'] = filename[:10]
        # change Date column into datetime object
        df_two['Date'] = pd.to_datetime(df_two['Date'])

        # group
        df_two['Group'] = temp.iloc[:, 1].str.strip()

        # create 14 more rows for each school (so we can have 15 grades)
        df_two = df_two.append([df_two]*14, ignore_index=True)

        # sort by school name and reset index
        df_two = df_two.sort_values(by=['School', 'Group'])
        df_two.reset_index(inplace=True, drop=True)

        for i in range(len(df_two)):

            # grades
            idx = i % 15
            df_two.loc[i, 'Grade'] = grades[idx]

        df_two['Enrolled'] = 0
        for i in range(len(df_two)):

            # enrollment
            school = df_two['School'][i]
            grade = str(df_two['Grade'][i])
            group = str(df_two['Group'][i])
            
            if not temp[(temp.iloc[:, 0].str.strip() == school) & (temp.iloc[:, 1].str.strip() == group)][grade].values.size == 0:
                    df_two.loc[i, 'Enrolled'] =temp[(temp.iloc[:, 0].str.strip() == school) & (temp.iloc[:, 1].str.strip() == group)][grade].values[0]
        
        df_two.dropna(inplace=True)

        # remove rows that aren't school (Total, )
        df_two = df_two[df_two['School'].str.contains('tota|campus', case=False, regex=True) != True]
        df_two.reset_index(inplace=True, drop=True)
        
        # change school names
        change_school_name(df_two)

        # set all schools to 'ES' (default)
        # then change types accordingly
        df_two['Type'] = 'ES'
        for i in range(len(df_two)):
            if df_two.loc[i, 'School'] in school_type.keys():
                df_two.loc[i, 'Type'] = school_type[df_two.loc[i, 'School']]
        
        
        # 2018-09-11, different format, change group and school name
        for i in range(len(df_two)):
            if df_two.loc[i, 'Group'] in group_dict.keys():
                df_two.loc[i, 'Group'] = group_dict[df_two.loc[i, 'Group']]
        
        #append each temp dataframes(sheets) to a list
        df_excel.append(df_two)

df_two_complete = pd.DataFrame(columns=['Date', 'School', 'Grade', 'Enrolled', 'Group', 'Type'])

for i in range(len(df_excel)):
    df_two_complete = df_two_complete.append(df_excel[i], sort=False, ignore_index=True)

2018-08-28 Enrollment Report.xls.xlsx
2018-09-04 Enrollment Report.xls.xlsx
2018-09-11 Enrollment Report.xls.xlsx
2018-09-18 Enrollment Report.xls.xlsx
2018-09-25 Enrollment Report.xls.xlsx
2018-10-02 Enrollment Report.xls.xlsx
2018-10-09 Enrollment Report.xls.xlsx
2018-10-16 Enrollment Report.xls.xlsx
2018-10-23 Enrollment Report.xls.xlsx
2018-10-30 Enrollment Report.xls.xlsx
2018-11-06 Enrollment Report.xls.xlsx
2018-11-16 Enrollment Report.xls.xlsx


In [9]:
for i in range(len(df_two_complete)):
    if df_two_complete.loc[i, 'Grade'] in grades_dict.keys():
        df_two_complete.loc[i, 'Grade'] = grades_dict[df_two_complete.loc[i, 'Grade']]

In [10]:
# file directories
# excel files after 2018-05-29 ~ 11/16
path_three = os.getcwd() + '/data/excel/three/'

# list of dataframes
# each dataframe is one excel file
df_excel = []

grades = ['EE', 'PK', 'KG', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# file names
filenames_three = os.listdir(path_three)
filenames_three.sort()

for filename in filenames_three:
    df_three = pd.DataFrame()

    # xlsx and xls file only
    if filename.endswith('xlsx' or 'xls'):
        print(filename)
        
        # get the first sheet in an excel file
        temp = pd.read_excel(path_three + filename, header=2, sheet_name=1)
        temp.iloc[:, 0].fillna(method='ffill', inplace=True)
        temp.iloc[:, 1:16].fillna(0, inplace=True)

        df_three = pd.DataFrame()

        # school name column 
        df_three['School'] = temp.iloc[:, 0].str.strip()

        # date column
        df_three['Date'] = filename[:10]
        # change Date column into datetime object
        df_three['Date'] = pd.to_datetime(df_three['Date'])

        # group
        df_three['Group'] = temp.iloc[:, 1].str.strip()

        # create 14 more rows for each school (so we can have 15 grades)
        df_three = df_three.append([df_three]*14, ignore_index=True)

        # sort by school name and reset index
        df_three = df_three.sort_values(by=['School', 'Group'])
        df_three.reset_index(inplace=True, drop=True)

        for i in range(len(df_three)):

            # grades
            idx = i % 15
            df_three.loc[i, 'Grade'] = grades[idx]

        df_three['Enrolled'] = 0
        for i in range(len(df_three)):

            # enrollment
            school = df_three['School'][i]
            grade = str(df_three['Grade'][i])
            group = str(df_three['Group'][i])
            
            if not temp[(temp.iloc[:, 0].str.strip() == school) & (temp.iloc[:, 1].str.strip() == group)][grade].values.size == 0:
                    df_three.loc[i, 'Enrolled'] =temp[(temp.iloc[:, 0].str.strip() == school) & (temp.iloc[:, 1].str.strip() == group)][grade].values[0]
        
        df_three.dropna(inplace=True)

        # remove rows that aren't school (Total, )
        df_three = df_three[df_three['School'].str.contains('tota|campus', case=False, regex=True) != True]
        df_three.reset_index(inplace=True, drop=True)
        
        # change school names
        change_school_name(df_three)

        # set all schools to 'ES' (default)
        # then change types accordingly
        df_three['Type'] = 'ES'
        for i in range(len(df_three)):
            if df_three.loc[i, 'School'] in school_type.keys():
                df_three.loc[i, 'Type'] = school_type[df_three.loc[i, 'School']]
        
        
        # 2018-09-11, different format, change group and school name
        for i in range(len(df_three)):
            if df_three.loc[i, 'Group'] in group_dict.keys():
                df_three.loc[i, 'Group'] = group_dict[df_three.loc[i, 'Group']]
        
        #append each temp dataframes(sheets) to a list
        df_excel.append(df_three)

df_three_complete = pd.DataFrame(columns=['Date', 'School', 'Grade', 'Enrolled', 'Group', 'Type'])

for i in range(len(df_excel)):
    df_three_complete = df_three_complete.append(df_excel[i], sort=False, ignore_index=True)

for i in range(len(df_three_complete)):
    if df_three_complete.loc[i, 'Grade'] in grades_dict.keys():
        df_three_complete.loc[i, 'Grade'] = grades_dict[df_three_complete.loc[i, 'Grade']]

2018-11-13 Enrollment Report.xls.xlsx
2018-11-28 Enrollment Report.xls.xlsx
2018-12-04 Enrollment Report.xls.xlsx
2018-12-11 Enrollment Report.xls.xlsx
2018-12-18 Enrollment Report.xls.xlsx


In [11]:
df = df_one_complete.append([df_two_complete, df_three_complete], sort=False)

In [12]:
df.sort_values(by=['Date', 'School'])

,Date,School,Grade,Enrolled,Group,Type
0,2016-08-23,Anderson Mill,ECE,0,Regular,ES
1,2016-08-23,Anderson Mill,PK,32,Regular,ES
2,2016-08-23,Anderson Mill,K,65,Regular,ES
3,2016-08-23,Anderson Mill,1,55,Regular,ES
4,2016-08-23,Anderson Mill,2,56,Regular,ES
5,2016-08-23,Anderson Mill,3,61,Regular,ES
6,2016-08-23,Anderson Mill,4,45,Regular,ES
7,2016-08-23,Anderson Mill,5,53,Regular,ES
8,2016-08-23,Anderson Mill,6,0,Regular,ES
9,2016-08-23,Anderson Mill,7,0,Regular,ES


In [18]:
df.reset_index(inplace=True, drop=True)

In [14]:
df['Enrolled'] = (df['Enrolled'].astype(int))

In [15]:
df.reset_index(drop=True, inplace=True)

In [16]:
df.sort_values(by=['Date', 'School', 'Group'], inplace=True)

In [19]:
df.to_csv('./excel_files.csv')

In [17]:
pdf = pd.read_csv('./pdf_1213.csv')

In [18]:
p_set = set(pdf['School'])

In [19]:
e_set = set(df['School'])

In [20]:
p_set - e_set

{'JJAEP LOTT'}

In [22]:
e_set - p_set

{'Early College', 'JJAEP', 'Johnson', 'LOTT', 'Pearson Ranch'}